# EXAMPLE 5
## Explaining the experiment
In this example, we are investigating the configuration memory of an FPGA with 25,611,008 bits (of which aroun 128k are flip-flops, the rest SRAM cells) organized in 800.344x32 words. A golden file following the manufacturer's rules was uploaded on the FPGA and then irradiated in static mode in an only round. Later, the content was downloaded and compared with the initial file.  

## Loading packages
The very first thing we must do is to load the packages required to load files (_DelimitedFiles_) as well as the LELAPE module. I suppose you have installed both. Load is done with:

In [ ]:
### Optional
push!(LOAD_PATH, "PATH_TO_FOLDER_WITH_LELAPE.jl");
#if the following cell is not successfully executed, run this one with the details of your system.

In [1]:
using DelimitedFiles, LELAPE

## Defining variables
Previous paragraph allows us to define several variables for checking the tests:

* Word width : 32 bits
* Memory size in words: 800344.
* In FPGAs, it seems more likely to succeed the POS operation.
* Tests were static. No information about cycles is necessary.

Ok, let us use this information to set these variables:

In [2]:
LA = 800344 # Memory size in words
WordWidth = 32 # Selfexplaining.
Operation = "POS" # Only "XOR" or "POS" are allowed.
KeepCycles = false # This is a Bool variable and only true false are accepted.

false

## Loading data
Results are stored in three different files following the required format: 
* CSV files 
* Every row is formed as WORD ADDRESS, READ VALUE, PATTERN, CYCLE.
Besides, the first row contains column heading (must be skipped), separators are commas and EOL character is the standard. 

We will use the _readdlm_ function provided by the _DelimitedFiles_ package to load the first CSV file and to store everything in the new variable, DATA. Finally, it is important to indicate that DATA must be an array of UInt32 numbers. 

In [4]:
DATA1 = readdlm("ExampleFPGA01.csv", ',', UInt32, '\n', skipstart=1)

124×4 Matrix{UInt32}:
 0x00000831  0x00000040  0x00000000  0x00000001
 0x00002f78  0x00008000  0x00000000  0x00000001
 0x00003849  0x00040000  0x00000000  0x00000001
 0x00007ebd  0x00000100  0x00000000  0x00000001
 0x00009332  0x00000300  0x00000000  0x00000001
 0x00009397  0x00000180  0x00000000  0x00000001
 0x00011fa0  0x00400000  0x00000000  0x00000001
 0x0001279d  0x00000200  0x00000000  0x00000001
 0x00012802  0x00000100  0x00000000  0x00000001
 0x00014d77  0x00000040  0x00000000  0x00000001
 0x00014ddc  0x00000020  0x00000000  0x00000001
 0x000154ea  0x00100000  0x00000000  0x00000001
 0x000180b1  0x00001000  0x00000000  0x00000001
          ⋮                          
 0x000add01  0x00000002  0x00000000  0x00000001
 0x000afbd9  0x00040000  0x00000000  0x00000001
 0x000b6fd1  0x000e0000  0x00000000  0x00000001
 0x000b7eb2  0x40000000  0x00000000  0x00000001
 0x000b954a  0x00004000  0x00000000  0x00000001
 0x000b975d  0x10000000  0x00000000  0x00000001
 0x000bb1d9  0x20000000  0x0

Good!! If you have correctly proceeded, a 124x4 unsigned integer matrix is loaded. Now, let us analyze the DATA1 but we need to define in advance several variables to set the analysis.

## Looking for MBUs
This analyisis is quite simple. We will call the _CheckMBUs_ function, which returns the MBUs present in DATA.Input arguments are the second and third columns, and the wordwidth.

This function returns two vectors. The first one indicates in position _k_ the number of bitflips observed in the _kth_ word. The second one is a vector of vectors and contains more detailed information: not only the number of bitflips per word but the position of the flipped bit (0 = LSB, WordWidth-1 = MSB). 

In [5]:
MBUSize, MBU_bit_pos = CheckMBUs(DATA1[:,2], DATA1[:,3], WordWidth)

([1, 1, 1, 1, 2, 2, 1, 1, 1, 1  …  3, 1, 1, 1, 1, 1, 1, 1, 1, 1], Any[[6], [15], [18], [8], [8, 9], [7, 8], [22], [9], [8], [6]  …  [17, 18, 19], [30], [14], [28], [29], [7], [12], [11], [16], [16]])

The following loop will show how many MBUs per number of flipped bits were observed:

In [8]:
for size = 1: WordWidth
    NMBUs = length(findall(MBUSize.==size))
    NMBUs!=0 ? println("$size-bit MBUs: ", NMBUs) : nothing
end

1-bit MBUs: 107
2-bit MBUs: 16
3-bit MBUs: 1


Interesting! There are some MBUs but... are they real or just the accumulation of SBUs. Let us perform an analysis of the presence of false 2- and 3-bit MBUs.

## Looking for MCUs
As modern memories are interleaved, it is not worth investigating MBUs but MCUs. Now, the system will combine addresses in all the possible pairs and operate them to create a DV set. If there were no MCUs, their characteristics are known. 

In particular, we can state that if the expected number of elements repeated _k_ times in this set is lower than a very low positive number, it is impossible to observe this number of repetitions unless the Only SBU assumption fails. We will define this threshold as 0.001 (default, 0.05). 


Although without a solid theoretical background, it seems that using pseudoaddress instead of word address provides better results.

Some experiments seem to show that if an element with very few number of 1s in binary format is too often repeated, it is indicative of the presence of MCUs. This is the Trace Rule and, in our analyisis, we want to keep all those too often repeated elements such that contain 2 ones or less in binary format.

Finally, perhaps we know that MCUs will not very large. For example, we may guess that MCUs with more than 20 bitflips are totally rejected. Therefore, to help the software and to avoid running out of memory, we will say the program _"Don't be silly and do not expect events larger than 20!!"_ If somehow this idea was wrong, we can change this value again and repeat the calculations.

In [6]:
ϵ = 0.001   # If the expected number of elements repeated k times is lower than ϵ, 
            # we can afirm that this is virtually impossible.
UsePseudoAddress = true
TraceRuleLength = 2
LargestMCUSize = 20

20

Time to test!!! We will call the function. Deppending on the set size or even if this is your first test, it will take you more or less time (Don't get up from your chair, though!!!!)

The following instruction will look for:
1. Values that pass the self-consistency test (C1_SCY)
2. Values found after inspecting MCUs derived from self-consistency-test (C1_MCU).
3. Values with less than or equal to _TraceRuleLength_ 1s in binary format that appear too often in the DV set (C1_TRC).
4. Values that, after combining in pairs the union of all the previous three sets and applying the operation and that appear too many times within the DV set (C1_SHF).

The first column of each matrix are the possible values and the second one the times it appeared.

In [11]:
C4_SCY, C4_MCU, C4_TRC, C4_SHF = DetectAnomalies_FullCheck(DATA4, WordWidth, LA, Operation, TraceRuleLength, UsePseudoAddress, KeepCycles, ϵ, LargestMCUSize)


(UInt32[0x00000800 0x00000012; 0x00400800 0x0000000d; 0x00700800 0x00000007], Matrix{UInt32}(undef, 0, 2), Matrix{UInt32}(undef, 0, 2), Matrix{UInt32}(undef, 0, 2))

Perhaps these matrices are hard to read since, for efficiency, they were returned in UInt32 format, even the number of occurrences!!! Execute the following instrucction for a better comprehension.

In [13]:
println("Elements appearing more than expected and passing the Self-Consistency test:\n")
for index in 1:length(C4_SCY[:, 1])
    println("Value: 0x", string(C4_SCY[index, 1], base=16, pad = 6), " --> ", Int(C4_SCY[index, 2]),".")
end

UsePseudoAddress ? L = LA*WordWidth : L = LA

print("\nOnly up to ", MaxExpectedRepetitions(NPairs(DATA4, UsePseudoAddress, WordWidth, KeepCycles), L, Operation, ϵ)-1, " repetitions are explained by randomness.")

Elements appearing more than expected and passing the Self-Consistency test:

Value: 0x000800 --> 18.
Value: 0x400800 --> 13.
Value: 0x700800 --> 7.

Only up to 4 repetitions are explained by randomness.

In this example, it is not worth to check the other sets since they did not yield any positive result. If you had had success, you would only have to do the following:

In [14]:
C4_All = [C4_SCY; C4_MCU; C4_TRC; C4_SHF]

3×2 Matrix{UInt32}:
 0x00000800  0x00000012
 0x00400800  0x0000000d
 0x00700800  0x00000007

## Grouping bitflips
Now, we have discovered those values relating pairs of pseudoaddresses. Now, let us go to group events in DATA. 

The first step consists in labeling all the pseudoaddresses and grouping their assigned indexes to a matrix containing information for the possible MCUs. It is an intermediate step and is done with the instruction _MCU_Indexes_ with the required and already defined parameters. 

In [15]:
Labeled_addresses = MCU_Indexes(DATA4, Operation, C4_All[:, 1], UsePseudoAddress, WordWidth)

38×2 Matrix{Int64}:
  20  240
  30  329
  33  334
  36  277
  41  339
  50  290
  52   54
  62  360
  64   65
  66  306
  70  308
  71  309
  77   78
   ⋮  
 171  172
 177  413
 194  195
 249  250
 274  275
 280  281
 284  286
 325  326
 342  343
 374  375
 379  380
 392  393

You should have got a 38x2 matrix so there are 38 2-bit MCUs.  

Now, we will classify addresses with _Classify_Addresses_in_MCU()_, using the previous matrix, _Labeled_addresses_. This function returns a vector of matrices. The first element is a matrix with pseudoaddresses related to 2-bit MCUs, and the second and last one to SBUs.

In [17]:
Events = Classify_Addresses_in_MCU(DATA4, Labeled_addresses, UsePseudoAddress, WordWidth)

2-element Vector{Any}:
 UInt32[0x00097d22 0x00497522; 0x00100674 0x00600e74; … ; 0x006cb133 0x006cb933; 0x006f72b7 0x006f7ab7]
 UInt32[0x00000f02, 0x00003d14, 0x0000b1b0, 0x0000bdd2, 0x0000cb10, 0x0000fbc4, 0x0001b5c5, 0x0001cfd7, 0x00023043, 0x000294d4  …  0x007ca960, 0x007d2a63, 0x007d4013, 0x007d5e23, 0x007e02e2, 0x007e0304, 0x007e5b74, 0x007f9112, 0x007f9b16, 0x007fcd23]

---------------------
Difficult to read, isn't it? The following instruction makes the content more readable:

In [18]:
for k = 1:length(Events) 
    NMCUs = length(Events[k][:, 1])
    println("Pseudoaddresses involved in $(length(Events)-k+1)-bit MCUs ($NMCUs events):")
    for row = 1:NMCUs
        for bit = 1:length(Events)-k+1
            print("0x", string(Events[k][row, bit], base=16, pad = 6), )
            
            bit != length(Events)-k+1 ? print(", ") : print("\n")

        end
    end
    println()
end

Pseudoaddresses involved in 2-bit MCUs (38 events):
0x097d22, 0x497522
0x100674, 0x600e74
0x116582, 0x616d82
0x11bb60, 0x51b360
0x12a146, 0x62a946
0x149e53, 0x549653
0x14d613, 0x14de13
0x17e2e6, 0x67eae6
0x17f3d7, 0x17fbd7
0x1808d4, 0x5800d4
0x186e44, 0x586644
0x1889d6, 0x5881d6
0x1a7393, 0x1a7b93
0x1ae1a6, 0x6ae9a6
0x1b7145, 0x1b7945
0x1b94e6, 0x6b9ce6
0x1d7684, 0x6d7e84
0x1df4f7, 0x1dfcf7
0x1ed6c3, 0x1edec3
0x1ed9e0, 0x5ed1e0
0x227787, 0x227f87
0x23ff34, 0x63f734
0x279e26, 0x679626
0x27fbf6, 0x67f3f6
0x2abcb0, 0x6ab4b0
0x2fceb0, 0x6fc6b0
0x34b4a7, 0x34bca7
0x363f02, 0x763702
0x3bb4d3, 0x3bbcd3
0x4b85d7, 0x4b8dd7
0x517195, 0x517995
0x5217b3, 0x521fb3
0x52e523, 0x52ed23
0x5f1123, 0x5f1923
0x63a093, 0x63a893
0x6c06a5, 0x6c0ea5
0x6cb133, 0x6cb933
0x6f72b7, 0x6f7ab7

Pseudoaddresses involved in 1-bit MCUs (361 events):
0x000f02
0x003d14
0x00b1b0
0x00bdd2
0x00cb10
0x00fbc4
0x01b5c5
0x01cfd7
0x023043
0x0294d4
0x030aa2
0x041a15
0x04b9f0
0x054d45
0x05df62
0x05f170
0x06bb52
0x072992
0x079586
0

## Are there false 2-bit MCUs?
The only-SBU Model allows estimating the number of false 2-bit events. We can use an implemented function in LELAPE:

In [27]:
NF2BIT = NF2BitMCUs(DATA4, LA, Operation, length(C4_All[:,1]), WordWidth, KeepCycles, UsePseudoAddress)
println("We expected $NF2BIT false 2-bit MCUs in this  experiment.")

We expected 0.01703488826751709 false 2-bit MCUs in this  experiment.


## Analysis completed
You can find in this folder other actual data got from the same memory. Analyze them and, if you wish, add aditional cells combining the anomalous DV values and try to do the most accurate example.